# Demonstration for Ohio Model User's Group

In [1]:
import os
import sys
import yaml

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import Project
from lasso import metcouncil

In [2]:
%load_ext autoreload
%autoreload 2

### Set up the Logger & Parameters

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.setLevel(logging.INFO)

### Input/Output Locations & Parameters

In [4]:
input_dir = os.path.join("Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities", "examples", "stpaul")
lasso_dir = os.path.join("Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities")
output_dir = input_dir

stpaul_shape_file = os.path.join(input_dir, "shape.geojson")
stpaul_link_file = os.path.join(input_dir, "link.json")
stpaul_node_file = os.path.join(input_dir, "node.geojson")

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-03-16 13:53:42, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities


### Data Reads

In [6]:
roadway_net = RoadwayNetwork.read(
    link_file = stpaul_link_file, 
    node_file = stpaul_node_file, 
    shape_file = stpaul_shape_file, 
    fast = True        
)

transit_net = TransitNetwork.read(feed_path = input_dir)

2021-03-16 13:53:42, INFO: Reading from following files:
-Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\examples\stpaul\link.json
-Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\examples\stpaul\node.geojson
-Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\examples\stpaul\shape.geojson.
2021-03-16 13:53:53, INFO: Read 66253 links from Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\examples\stpaul\link.json
2021-03-16 13:53:53, INFO: Read 17159 nodes from Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\examples\stpaul\node.geojson
2021-03-16 13:53:53, INFO: Read 66253 shapes from Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\examples\stpaul\shape.geojson
2021-03-16 13:54:05, INFO: Read in transit feed from: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\examples\stpaul
2021-03-16 1

In [7]:
roadway_net.links_df.columns

Index(['model_link_id', 'osm_link_id', 'shstReferenceId', 'shstGeometryId',
       'shape_id', 'u', 'v', 'A', 'B', 'locationReferences', 'distance',
       'roadway', 'name', 'ref', 'bridge', 'tunnel', 'width', 'max_speed',
       'bike_facility', 'drive_access', 'walk_access', 'bike_access',
       'truck_access', 'bus_only', 'rail_only', 'lanes', 'access', 'price',
       'trn_priority', 'ttime_assert', 'geometry'],
      dtype='object')

### Make a Scenario 
We manipulate scenarios

In [8]:
base_scenario = Scenario.create_scenario(base_scenario = {"road_net": roadway_net, "transit_net": transit_net})

2021-03-16 13:54:05, INFO: Creating Scenario


### Make a Cube Network

In [9]:
base_scenario.road_net.links_df = base_scenario.road_net.links_df[base_scenario.road_net.links_df['drive_access'] == 1]

model_road_net = ModelRoadwayNetwork.from_RoadwayNetwork(base_scenario.road_net)
model_road_net = metcouncil.calculate_centroidconnect(model_road_net, parameters)
model_road_net.roadway_standard_to_met_council_network()

2021-03-16 13:54:07, INFO: Lasso base directory set as: z:\Data\Users\Sijia\Met_Council\github\client_met_council_wrangler_utilities
2021-03-16 13:54:07, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2021-03-16 13:54:07, INFO: Finished calculating centroid connector variable: centroidconnect
2021-03-16 13:54:07, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2021-03-16 13:54:07, INFO: Didn't detect managed lanes in network.
2021-03-16 13:54:07, INFO: Creating calculated roadway variables.
2021-03-16 13:54:07, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type


C:\Users\wangs1\.conda\envs\lasso\lib\site-packages\geopandas\tools\sjoin.py:56: UserWarning: CRS of frames being joined does not match!(4326 != {'init': 'epsg:4326', 'no_defs': True})
  '(%s != %s)' % (left_df.crs, right_df.crs))


2021-03-16 13:54:08, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2021-03-16 13:54:08, INFO: Adding roadway network variable for county using a spatial join with: z:\Data\Users\Sijia\Met_Council\github\client_met_council_wrangler_utilities\metcouncil_data\county\cb_2017_us_county_5m.shp
2021-03-16 13:54:10, INFO: Finished Calculating county variable: county
2021-03-16 13:54:10, INFO: Calculating MPO as roadway network variable: mpo
2021-03-16 13:54:10, INFO: Finished calculating MPO variable: mpo
2021-03-16 13:54:10, INFO: Adding Counts
2021-03-16 13:54:10, INFO: Adding Variable AADT using Shared Streets Reference from z:\Data\Users\Sijia\Met_Council\github\client_met_council_wrangler_utilities\metcouncil_data\count_mn\mn_count_ShSt_API_match.csv
2021-03-16 13:54:10, INFO: Added variable: AADT using Shared Streets Reference
2021-03-16 13:54:10, INFO: Adding Variable AADT using Shared Streets Reference from z:\Data\Users\Sijia\Met_Council\github\client

In [10]:
# model_road_net.write_roadway_as_fixedwidth(
#     output_link_txt = os.path.join(output_dir, "links.txt"),
#    output_node_txt = os.path.join(output_dir, "nodes.txt"),
#    output_link_header_width_txt = os.path.join(output_dir, "links_header_width.txt"),
#    output_node_header_width_txt = os.path.join(output_dir, "nodes_header_width.txt"),
#    output_cube_network_script = os.path.join(output_dir, "make_complete_network_from_fixed_width_file.s"),
#)

In [11]:
#standard_transit = StandardTransit.fromTransitNetwork(transit_net)
#standard_transit.write_as_cube_lin(outpath = os.path.join(output_dir, "transit.lin") )

### Make a Project Card from a Cube Log File

In [12]:
project = Project.create_project(
        base_roadway_network = model_road_net,
        roadway_log_file = os.path.join(lasso_dir, "examples", "stpaul", "cube", "st_paul_example.log"),
        project_name = "Example",
    )

2021-03-16 13:54:12, INFO: No base transit network.
2021-03-16 13:54:12, INFO: No transit changes given or processed.
2021-03-16 13:54:12, INFO: Reading logfile: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\examples\stpaul\cube\st_paul_example.log
2021-03-16 13:54:12, INFO: Processed 0 Node lines and 1 Link lines
2021-03-16 13:54:12, INFO: Lasso base directory set as: z:\Data\Users\Sijia\Met_Council\github\client_met_council_wrangler_utilities
2021-03-16 13:54:12, INFO: Evaluating compatibility between roadway network changes and base network. Not evaluating deletions.
2021-03-16 13:54:12, INFO: Evaluating project changes.
2021-03-16 13:54:12, WARNING: The following attributes are specified in the changes but do not exist in the base network: ['OPERATION_final']
2021-03-16 13:54:12, INFO: Lasso base directory set as: z:\Data\Users\Sijia\Met_Council\github\client_met_council_wrangler_utilities
2021-03-16 13:54:12, INFO: Lasso base directory set as: z:\Dat

z:\data\users\sijia\met_council\github\client_met_council_wrangler_utilities\lasso\project.py:613: RuntimeWarning: divide by zero encountered in double_scalars
  / base_row[col].astype(float)


In [13]:
project.write_project_card(os.path.join(output_dir, "ohio_mug", "Example.yml"))

2021-03-16 13:54:12, INFO: Wrote project card to: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\examples\stpaul\ohio_mug\Example.yml


### Apply the Project Card

In [16]:
#build_scenario = Scenario.create_scenario(base_scenario = {"road_net": roadway_net, "transit_net": transit_net},
#                                        card_directory = os.path.join(lasso_dir, "examples", "stpaul", "ohio_mug"),
#                                          #tags = ['checked'],
#                                          validate_project_cards = False)

In [17]:
#build_scenario.apply_all_projects()

In [14]:
build_scenario = Scenario.create_scenario(base_scenario = {"road_net": roadway_net, "transit_net": transit_net})

2021-03-16 13:54:13, INFO: Creating Scenario


In [15]:
build_scenario.apply_project(project.card_data)

2021-03-16 13:54:13, INFO: Applying Example
2021-03-16 13:54:13, INFO: Applying Example
2021-03-16 13:54:13, INFO: Applying Project to Roadway Network: Example


z:\data\users\sijia\met_council\github\client_met_council_wrangler_utilities\lasso\src\network-wrangler\network_wrangler\roadwaynetwork.py:870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_links["i"] = 0


In [16]:
build_scenario.applied_projects

['Example']

In [17]:
project.card_data

{'project': 'Example',
 'changes': [{'category': 'Roadway Property Change',
   'facility': {'link': [{'model_link_id': [389369]}]},
   'properties': [{'existing': 0.0, 'set': 0.1251, 'property': 'distance'},
    {'existing': 2, 'set': 3, 'property': 'lanes'}],
   'project': 'Example'}]}

In [18]:
build_scenario.road_net.links_df[build_scenario.road_net.links_df.model_link_id == 389369].lanes

7588    3
Name: lanes, dtype: int64

### Write Out Edited Cube Network

In [27]:
model_road_net = ModelRoadwayNetwork.from_RoadwayNetwork(build_scenario.road_net)
model_road_net.roadway_standard_to_met_council_network()

2021-03-15 18:21:47, INFO: Lasso base directory set as: z:\Data\Users\Sijia\Met_Council\github\client_met_council_wrangler_utilities
2021-03-15 18:21:47, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2021-03-15 18:21:47, INFO: Didn't detect managed lanes in network.
2021-03-15 18:21:47, INFO: Creating calculated roadway variables.
2021-03-15 18:21:47, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type


C:\Users\wangs1\.conda\envs\lasso\lib\site-packages\geopandas\tools\sjoin.py:56: UserWarning: CRS of frames being joined does not match!(4326 != {'init': 'epsg:4326', 'no_defs': True})
  '(%s != %s)' % (left_df.crs, right_df.crs))


2021-03-15 18:21:56, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2021-03-15 18:21:56, INFO: Adding roadway network variable for county using a spatial join with: z:\Data\Users\Sijia\Met_Council\github\client_met_council_wrangler_utilities\metcouncil_data\county\cb_2017_us_county_5m.shp
2021-03-15 18:22:01, INFO: Finished Calculating county variable: county
2021-03-15 18:22:01, INFO: Calculating MPO as roadway network variable: mpo
2021-03-15 18:22:01, INFO: Finished calculating MPO variable: mpo
2021-03-15 18:22:01, INFO: Adding Counts
2021-03-15 18:22:01, INFO: Adding Variable AADT using Shared Streets Reference from z:\Data\Users\Sijia\Met_Council\github\client_met_council_wrangler_utilities\metcouncil_data\count_mn\mn_count_ShSt_API_match.csv
2021-03-15 18:22:02, INFO: Added variable: AADT using Shared Streets Reference
2021-03-15 18:22:02, INFO: Adding Variable AADT using Shared Streets Reference from z:\Data\Users\Sijia\Met_Council\github\client

In [29]:
model_road_net.links_metcouncil_df[model_road_net.links_metcouncil_df.model_link_id == 389369]["lanes_AM"]

7588    3
Name: lanes_AM, dtype: int32

In [28]:
model_road_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, "links.txt"),
    output_node_txt = os.path.join(output_dir, "nodes.txt"),
    output_link_header_width_txt = os.path.join(output_dir, "links_header_width.txt"),
    output_node_header_width_txt = os.path.join(output_dir, "nodes_header_width.txt"),
    output_cube_network_script = os.path.join(output_dir, "make_complete_network_from_fixed_width_file.s"),
)

2021-03-15 18:22:56, INFO: Starting fixed width conversion
2021-03-15 18:24:25, INFO: Writing out link database
2021-03-15 18:24:28, INFO: Writing out link header and width ----
2021-03-15 18:24:28, INFO: Starting fixed width conversion
2021-03-15 18:24:31, INFO: Writing out node database
2021-03-15 18:24:31, INFO: Writing out node header and width
